In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Change numerical features to %

In [2]:
## Load .csv

df= pd.read_csv('../raw_data/enriched_CT_PM.csv')

In [3]:
df_numerical= df.select_dtypes(exclude='object')

In [4]:
df_numerical.head()

,Unnamed: 0,Authorisation status,Generic,Biosimilar,Orphan medicine,n_trials,status_not_yet_recruiting,status_recruiting,status_enrolling_by_invitation,status_active_not_recruiting,...,org_nih,org_other,org_other_gov,phase_early_1,phase_not_applicable,phase_1,phase_2,phase_3,phase_4,pm_results
0,0,0,0,0,0,54,0,2,0,1,...,0,12,2,0,2,1,2,17,30,44
1,1,0,0,0,1,12,0,2,0,1,...,0,2,0,0,0,1,4,3,0,11
2,2,0,1,0,0,20,0,0,0,0,...,0,3,0,0,2,0,0,5,12,36
3,3,0,0,0,0,111,3,8,0,7,...,13,53,4,1,0,34,65,21,0,523
4,4,0,0,0,0,20,0,0,0,0,...,0,3,0,0,2,0,0,5,12,36


In [5]:
df_numerical.drop(labels= 'Unnamed: 0', inplace=True, axis=1)

c:\users\lucia\.venvs\lewagon\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [6]:
df_numerical.head()

,Authorisation status,Generic,Biosimilar,Orphan medicine,n_trials,status_not_yet_recruiting,status_recruiting,status_enrolling_by_invitation,status_active_not_recruiting,status_suspended,...,org_nih,org_other,org_other_gov,phase_early_1,phase_not_applicable,phase_1,phase_2,phase_3,phase_4,pm_results
0,0,0,0,0,54,0,2,0,1,0,...,0,12,2,0,2,1,2,17,30,44
1,0,0,0,1,12,0,2,0,1,0,...,0,2,0,0,0,1,4,3,0,11
2,0,1,0,0,20,0,0,0,0,0,...,0,3,0,0,2,0,0,5,12,36
3,0,0,0,0,111,3,8,0,7,0,...,13,53,4,1,0,34,65,21,0,523
4,0,0,0,0,20,0,0,0,0,0,...,0,3,0,0,2,0,0,5,12,36


In [7]:
percent_list= df_numerical.columns[5:-1]

In [8]:
percent_list

Index(['status_not_yet_recruiting', 'status_recruiting',
       'status_enrolling_by_invitation', 'status_active_not_recruiting',
       'status_suspended', 'status_terminated', 'status_completed',
       'status_withdrawn', 'status_unknown', 'org_fed', 'org_indiv',
       'org_industry', 'org_network', 'org_nih', 'org_other', 'org_other_gov',
       'phase_early_1', 'phase_not_applicable', 'phase_1', 'phase_2',
       'phase_3', 'phase_4'],
      dtype='object')

In [9]:
df_numerical['status_recruiting']=df_numerical['status_recruiting']/df_numerical['n_trials']

c:\users\lucia\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
for column in percent_list:
    df_numerical[column]=((df_numerical[column]*100)/df_numerical['n_trials']).replace([np.inf, -np.inf, np.nan], 0)

c:\users\lucia\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
def percentage_columns(df,column_list,column_total):
    for column in column_list:
        df[column]=((df[column]*100)/df[column_total]).replace([np.inf, -np.inf, np.nan], 0)
    return df

In [12]:
df_cleaned=df_numerical[df_numerical['n_trials']< 3000]

In [21]:
X= df_cleaned.drop(labels='Authorisation status', axis=1)
y=df_cleaned['Authorisation status']

In [22]:
from sklearn.feature_selection import SelectKBest, chi2

X_new = SelectKBest(chi2, k=10).fit_transform(X,y)

In [23]:
X_new.shape

(1220, 10)

In [24]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler().fit(X_new)
X_new= scaler.transform(X_new)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_new,y, test_size= 0.2, random_state=1)

In [26]:
from sklearn.neighbors import KNeighborsClassifier

knn= KNeighborsClassifier()
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.9590163934426229

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn import metrics


cv_results= cross_validate(knn, X_train, y_train, cv=5,
                          scoring=['accuracy','f1_macro'])


In [31]:
#print('Train f1:', cv_results['train_f1_macro'].mean())
#print('Train accuracy:', {cv_results['train_accuracy'].mean()})


print(f'Test f1:',{cv_results['test_f1_macro'].mean()})
print(f'Test accuracy:', {cv_results['test_accuracy'].mean()})

Test f1: {0.4900728325108562}
Test accuracy: {0.9610675039246468}


In [ ]:
df_numerical.head(1)

# Mierda

In [ ]:
sns.heatmap(df_numerical.corr())

In [ ]:
df.columns

In [ ]:
group = df.groupby(['Authorisation status','Generic']).size().unstack(level=1)
group.plot(kind = 'barh',stacked = True)

In [ ]:
group = df.groupby(['Authorisation status','Orphan medicine']).size().unstack(level=1)
group.plot(kind = 'barh',stacked = True)

In [ ]:
sns.catplot(data = df, x = 'Authorisation status',y = 'Orphan medicine', kind = 'box')

In [ ]:
for column in percent_list:
    sns.catplot(data = df_cleaned, x = 'Authorisation status',y = column, kind = 'box')

# Model 2

In [32]:
len(percent_list)
top_list= ['phase_4', 'Orphan medicine', 
           'org_indiv', 'n_trials', 
           'status_terminated', 'phase_2', 
           'phase_3', 'status_not_yet_recruiting', 
           'org_fed', 'org_nih','Authorisation status']

In [40]:
df_secondmodel= df_cleaned[top_list]

In [41]:
X2= df_secondmodel.drop(labels='Authorisation status', axis=1)
y2=df_secondmodel['Authorisation status']

In [42]:
from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test= train_test_split(X2,y2, test_size= 0.2, random_state=1)

In [48]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X2_train)
X2_train= scaler.transform(X2_train)

In [49]:
from sklearn.neighbors import KNeighborsClassifier

knn= KNeighborsClassifier()
knn.fit(X2_train,y2_train)
y2_pred= knn.predict(X2_train)

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y2_train, y2_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       938
           1       0.00      0.00      0.00        38

    accuracy                           0.96       976
   macro avg       0.48      0.50      0.49       976
weighted avg       0.92      0.96      0.94       976



c:\users\lucia\.venvs\lewagon\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
